In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pltB
import warnings
from    sklearn.model_selection import train_test_split
from    sklearn.metrics import confusion_matrix, classification_report
from    sklearn.ensemble import RandomForestClassifier
from    sklearn.preprocessing import LabelEncoder
from    sklearn.model_selection import GridSearchCV
from    sklearn.preprocessing import StandardScaler
from    sklearn.preprocessing import MinMaxScaler

from    imblearn.over_sampling import SMOTE
from    imblearn.under_sampling import RandomUnderSampler
from    imblearn.pipeline import Pipeline
from    imblearn.combine import SMOTETomek

warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'sklearn'

In [2]:
df = pd.read_csv("predictive_maintenance.csv")
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [3]:
df.describe()

,UDI,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,300.004930,310.005560,1538.776100,39.986910,107.951000,0.033900
std,2886.89568,2.000259,1.483734,179.284096,9.968934,63.654147,0.180981
min,1.00000,295.300000,305.700000,1168.000000,3.800000,0.000000,0.000000
25%,2500.75000,298.300000,308.800000,1423.000000,33.200000,53.000000,0.000000
50%,5000.50000,300.100000,310.100000,1503.000000,40.100000,108.000000,0.000000
75%,7500.25000,301.500000,311.100000,1612.000000,46.800000,162.000000,0.000000
max,10000.00000,304.500000,313.800000,2886.000000,76.600000,253.000000,1.000000


In [4]:
df=df.drop(['Product ID','UDI','Target'],axis=1)

Feature Engineering

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'sklearn'

In [17]:
label_encoder = LabelEncoder()

# Perform label encoding
df['Failure Type_encoded'] = label_encoder.fit_transform(df['Failure Type'])

df['Failure Type_encoded'].value_counts()

Failure Type_encoded
1    9652
0     112
3      95
2      78
5      45
4      18
Name: count, dtype: int64

Failure Type_encoded
1    9652 - no Failure
0     112 - Heat Dissipation
3      95 - Power  Failure
2      78 - Overstrain Failure
5      45 - Tool Wear Failure
4      18 - Random Failures

In [18]:
df = pd.get_dummies(df, columns=['Type'], prefix='Type', drop_first=True)
df.head()



,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,Failure Type_encoded,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,No Failure,1,False,True
1,298.2,308.7,1408,46.3,3,No Failure,1,True,False
2,298.1,308.5,1498,49.4,5,No Failure,1,True,False
3,298.2,308.6,1433,39.5,7,No Failure,1,True,False
4,298.2,308.7,1408,40.0,9,No Failure,1,True,False


In [19]:
df[50:52]

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,Failure Type_encoded,Type_L,Type_M
50,298.9,309.1,2861,4.6,143,Power Failure,3,True,False
51,298.9,309.1,1383,54.9,145,No Failure,1,True,False


In [20]:
from imblearn.combine import SMOTETomek

# Define features and target
X = df.drop(['Failure Type','Failure Type_encoded'], axis=1).values
y = df['Failure Type_encoded'].values

# Split data into train, validation, and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

# Count the original class distribution
print("Class distribution before balancing:")
print(df['Failure Type_encoded'].value_counts())


smt=SMOTETomek(sampling_strategy='auto',random_state=42)

X_trainresampled, y_trainresampled = smt.fit_resample(X_train, y_train)

# Convert the resampled dataset to a DataFrame
df_resampled = pd.DataFrame(X_trainresampled, columns=[f'feature_{i}' for i in range(X_trainresampled.shape[1])])
df_resampled['Failure Type_encoded'] = y_trainresampled

# Count the class distribution after balancing
print("\nClass distribution after balancing:")
print(df_resampled['Failure Type_encoded'].value_counts())

Class distribution before balancing:
Failure Type_encoded
1    9652
0     112
3      95
2      78
5      45
4      18
Name: count, dtype: int64

Class distribution after balancing:
Failure Type_encoded
2    7716
5    7712
3    7710
0    7708
4    7708
1    7688
Name: count, dtype: int64


Failure Type_encoded
1    9652 - no Failure
0     112 - Heat Dissipation
3      95 - Power  Failure
2      78 - Overstrain Failure
5      45 - Tool Wear Failure
4      18 - Random Failures

In [21]:
df_resampled.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,Failure Type_encoded
0,298.3,309.1,1616.0,31.1,195.0,1.0,0.0,1
1,298.2,308.4,1388.0,53.8,137.0,1.0,0.0,1
2,298.2,307.8,1528.0,31.1,194.0,1.0,0.0,1
3,300.9,310.8,1599.0,33.0,7.0,0.0,1.0,1
4,301.4,310.5,1571.0,33.9,208.0,1.0,0.0,1


In [22]:
df_resampled = pd.get_dummies(df_resampled, columns=['Failure Type_encoded'], prefix='Failure', drop_first=False)
df_resampled.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,Failure_0,Failure_1,Failure_2,Failure_3,Failure_4,Failure_5
0,298.3,309.1,1616.0,31.1,195.0,1.0,0.0,False,True,False,False,False,False
1,298.2,308.4,1388.0,53.8,137.0,1.0,0.0,False,True,False,False,False,False
2,298.2,307.8,1528.0,31.1,194.0,1.0,0.0,False,True,False,False,False,False
3,300.9,310.8,1599.0,33.0,7.0,0.0,1.0,False,True,False,False,False,False
4,301.4,310.5,1571.0,33.9,208.0,1.0,0.0,False,True,False,False,False,False


In [23]:
df_resampled.value_counts()

feature_0   feature_1   feature_2    feature_3  feature_4   feature_5  feature_6  Failure_0  Failure_1  Failure_2  Failure_3  Failure_4  Failure_5
295.300000  305.700000  1473.000000  38.500000  189.000000  0.000000   1.000000   False      True       False      False      False      False        1
301.753811  311.402556  1227.564126  67.353811  193.205385  0.282063   0.717937   False      False      True       False      False      False        1
301.754220  309.900000  1326.217475  55.506453  184.288978  0.457796   0.542204   True       False      False      False      False      False        1
301.754301  310.596249  1614.340807  29.425479  234.400734  0.940073   0.000000   False      False      False      False      False      True         1
301.754333  309.662999  1301.667476  62.729018  226.147519  1.000000   0.000000   True       False      False      False      False      False        1
                                                                                             

In [24]:
X = df_resampled.drop(['Failure_0', 'Failure_1', 'Failure_2',
       'Failure_3', 'Failure_4', 'Failure_5'], axis="columns")

y = df_resampled[['Failure_0', 'Failure_1', 'Failure_2',
       'Failure_3', 'Failure_4', 'Failure_5']]

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=21)

X_train.shape

(36993, 7)

In [26]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.metrics import accuracy_score


In [27]:
model = keras.Sequential()
model.add(layers.Dense(units=10, activation='relu', input_shape=(7,)))
model.add(layers.Dense(units=10, activation='relu'))
model.add(layers.Dense(units=10, activation='relu'))
model.add(layers.Dense(units=6, activation='softmax'))

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train,epochs=50)

Epoch 1/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 530us/step - accuracy: 0.8335 - loss: 0.4414
Epoch 2/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 573us/step - accuracy: 0.8342 - loss: 0.4349
Epoch 3/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 514us/step - accuracy: 0.8330 - loss: 0.4407
Epoch 4/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 531us/step - accuracy: 0.8358 - loss: 0.4343
Epoch 5/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 513us/step - accuracy: 0.8297 - loss: 0.4421
Epoch 6/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 514us/step - accuracy: 0.8350 - loss: 0.4262
Epoch 7/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 531us/step - accuracy: 0.8433 - loss: 0.4169
Epoch 8/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 513us/step - accuracy: 0.8323 - loss: 0.4346
Epoch 9/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 516us/step - accuracy: 0.8394 - loss: 0.4163
Epoch 10/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 520us/step - accuracy: 0.8400 - loss: 0.4177
Epoch 11/50
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 1s 518us/step - accuracy: 0.8389 - loss: 0.4125
Epoch 12

In [32]:
model.evaluate(X_test,y_test)

290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.8392 - loss: 0.4170


[0.43456292152404785, 0.8352254033088684]

In [33]:
summary = model.summary()
print(summary)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 10)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 6)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,100 (4.30 KB)

 Trainable params: 366 (1.43 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 734 (2.87 KB)

None


In [25]:
model.save("predmaint_v003.h5")

In [ ]:
Weights_Layer_0 = model.layers[0].get_weights()[0]
Weights_Layer_1 = model.layers[1].get_weights()[0]
Weights_Layer_2 = model.layers[2].get_weights()[0]
Weights_Layer_3 = model.layers[3].get_weights()[0]

Bias_Layer_0 = model.layers[0].get_weights()[1]
Bias_Layer_1 = model.layers[1].get_weights()[1]

print(Weights_Layer_0)
print(Bias_Layer_0)

#print(Weights_Layer_1)
#print(Bias_Layer_1)


In [1]:
import snap7
import ctypes
from snap7.types import Areas, S7DataItem, S7WLWord, S7WLReal, S7WLTimer
from snap7.util import set_int, set_real, get_int, get_real, get_s5time


ModuleNotFoundError: No module named 'snap7.types'

In [38]:
def read_real_values(client, db_number, start_address, number_values):

    values = []

    try:
        # Conectando ao PLC
        #client.connect(plc_ip, 0, 1)

            # Lendo  valores reais (floats) do DB
        data = client.db_read(db_number, start_address, number_values*4)  # 4 bytes para cada valor real

            # Convertendo os bytes em floats
        values = [snap7.util.get_real(data, i) for i in range(0, number_values*4, 4)]

    except Snap7Exception as e:
        print(f"Erro ao ler dados do PLC: {e}")    
        
    return values

In [39]:

def write_data_to_plc(client, dbnumber,offset, value):

    realValue = bytearray(4)
    
    set_real(realValue, 0,value)

    client.db_write(dbnumber,offset,realValue)



In [40]:
plc_ip = '192.168.0.1'
db_number = 2
start_address = 0
read_number_of_values = 7


In [44]:
s71511T = snap7.client.Client()
s71511T.connect(plc_ip, 0, 1)

if  s71511T.get_connected() == True:
    
    while True:
        values=read_real_values(s71511T,2,0,7)

        features = np.array(values)
        features = features.reshape(1,7)

        dframe = pd.DataFrame(features)

        predicted =  model.predict(dframe)

        write_data_to_plc(s71511T,3,0,predicted[0][0])
        write_data_to_plc(s71511T,3,4,predicted[0][1])
        write_data_to_plc(s71511T,3,8,predicted[0][2])
        write_data_to_plc(s71511T,3,12,predicted[0][3])
        write_data_to_plc(s71511T,3,16,predicted[0][4])
        write_data_to_plc(s71511T,3,20,predicted[0][5])
else:
    print("Sem conexão com o CLP")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

b' ISO : An error occurred during recv TCP : Connection timed out'


RuntimeError: b' ISO : An error occurred during recv TCP : Connection timed out'